<a href="https://colab.research.google.com/github/siona-pathak/DS2002-HW/blob/main/API_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import pprint
import requests.exceptions
import pandas as pd

In [ ]:
api_url = "https://restcountries.com/v3.1/name/"
data_file_path = "country_info.json"

def main():
    while True:
        name_of_country = input("Country Name (or enter 'quit' to Quit): ").strip().lower()
        if name_of_country == 'quit':
            break
        try:
            country_data = fetch_country_data(name_of_country)
            if country_data:
                display_country_info(country_data)
                append_data_to_json(country_data)
            else:
                print("Country not found or an error occurred. Please try again.")
        except requests.exceptions.HTTPError as errh:
            print("An HTTP Error occurred:", repr(errh))
        except requests.exceptions.ConnectionError as errc:
            print("An Error Connecting To The API Occurred:", repr(errc))
        except requests.exceptions.Timeout as errt:
            print("A Timeout Error occurred:", repr(errt))
        except requests.exceptions.RequestException as err:
            print("An Unknown Error occurred:", repr(err))

def return_country_data(name_of_country):
    url = api_url + name_of_country
    response = requests.get(url)
    response.raise_for_status()
    countries = response.json()
    for country in countries:
        if any(name_of_country == name.lower() for name in [country['name']['common'], country['name']['official']]):
            return {
                'Country': country['name']['common'],
                'Capital': country.get('capital', ['No Capital'])[0],
                'Population': country.get('population', 'N/A')
            }
    return None


def append_data_to_json(country_data):
    try:
        existing_df = pd.read_json(data_file_path)
    except (ValueError, FileNotFoundError):
        existing_df = pd.DataFrame()

    new_df = pd.DataFrame([country_data])
    updated_df = pd.concat([existing_df, new_df], ignore_index=True)
    updated_df.to_json(data_file_path, orient='records')

def display_country_info(country_data):
    print(f"Country: {country_data['Country']}")
    print(f"Capital: {country_data['Capital']}")
    print(f"Population: {country_data['Population']}")

if __name__ == "__main__":
     main()

